# Folded Cascode OTA Sizing using gm/ID (SG13G2 HV Devices)

In [1]:
import numpy as np
from pygmid import Lookup as lk

nmos = lk('sg13_hv_nmos.mat')
pmos = lk('sg13_hv_pmos.mat')

## System Specifications → Required gm

In [2]:
C_load = 3e-12     # Load capacitance from SC-CMFB
UGB = 5e6           # Target unity gain bandwidth

gm = 2*np.pi*UGB*C_load
print("Required gm = %.2f uS" % (gm*1e6))

Required gm = 94.25 uS


## gm/ID Design Choices

In [3]:
gmid_in   = 15   # Input pair
gmid_casc = 12   # Cascode devices
gmid_sink = 8    # Bottom current sinks
gmid_src  = 10   # PMOS top sources
gmid_tail = 8    # Tail current source

## Currents

In [4]:
Id_tail = gm / gmid_in
Id_branch = Id_tail / 2

print("Tail current = %.2f uA" % (Id_tail*1e6))
print("Branch current = %.2f uA" % (Id_branch*1e6))

Tail current = 6.28 uA
Branch current = 3.14 uA


## Length Choices

In [5]:
L_in = 1.0
L_tail = 1.5
L_casc_n = 0.45
L_casc_p = 0.4
L_sink = 1.0
L_src = 1.0

## Width Calculations

In [6]:
jd_in = nmos.lookup('ID_W', GM_ID=gmid_in, L=L_in)
W_in = Id_branch / jd_in

jd_tail = nmos.lookup('ID_W', GM_ID=gmid_tail, L=L_tail)
W_tail = Id_tail / jd_tail

jd_casc_n = nmos.lookup('ID_W', GM_ID=gmid_casc, L=L_casc_n)
W_casc_n = Id_branch / jd_casc_n

jd_casc_p = pmos.lookup('ID_W', GM_ID=gmid_casc, L=L_casc_p)
W_casc_p = Id_branch / jd_casc_p

jd_sink = nmos.lookup('ID_W', GM_ID=gmid_sink, L=L_sink)
W_sink = Id_branch / jd_sink

jd_src = pmos.lookup('ID_W', GM_ID=gmid_src, L=L_src)
W_src = Id_branch / jd_src

print("W_in =", W_in, "um")
print("W_tail =", W_tail, "um")
print("W_casc_n =", W_casc_n, "um")
print("W_casc_p =", W_casc_p, "um")
print("W_sink =", W_sink, "um")
print("W_src =", W_src, "um")

W_in = 6.195667238264645 um
W_tail = 3.2470449766871985 um
W_casc_n = 1.0055068472609252 um
W_casc_p = 4.615575616988637 um
W_sink = 1.1285814987908882 um
W_src = 7.047799650367065 um


## Headroom Check

In [7]:
Vds_stack = 2/gmid_casc + 2/gmid_casc + 2/gmid_sink
print("Estimated Vdsat stack =", Vds_stack, "V")

Estimated Vdsat stack = 0.5833333333333333 V
